In [ ]:
import os

In [1]:
directory = "results/classifications/20210716/"
suffix= '20210716'

### Rename files

In [ ]:
# for file in os.listdir(directory):
#     #print(file[0:58]+'20201215.tif')
#     os.rename(directory+file, directory+file[0:58]+'20201215.tif')

## Mosiac tifs

In [ ]:
#build vrt
os.chdir(directory)
os.system("gdalbuildvrt " + "Southern_gm_mads_two_seasons_"+suffix+"_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       "-co NUM_THREADS=14 "\
       "--config GDAL_CACHEMAX 512 "
       + "Southern_gm_mads_two_seasons_"+suffix+"_mosaic.vrt " + "Southern_gm_mads_two_seasons_"+suffix+"_mosaic.tif")

## Clip mosaic to AEZ

In [ ]:
# !pip install -e ../../production/dea_ml

In [2]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog
import sys

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:46871 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


In [4]:
# aez_region = 'data/Southern.shp'
aez_region = 'WDPA_southern.shp'
pred_tif = "results/classifications/"+suffix+"/Southern_gm_mads_two_seasons_"+suffix+"_mosaic.tif"
chunks={'x':5000, 'y':5000}

In [5]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [6]:
aez = xr_rasterize(aez, da)

/env/lib/python3.6/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [7]:
da = da.chunk(chunks).where(aez.chunk(chunks)).astype(np.int8)

In [8]:
%%time
da = da.compute()

CPU times: user 26.8 s, sys: 1min 1s, total: 1min 28s
Wall time: 2min 35s


In [9]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_clipped.tif',
          overwrite=True)

PosixPath('results/classifications/20210716/Southern_gm_mads_two_seasons_20210716_mosaic_clipped.tif')

In [7]:
aez

<xarray.DataArray (y: 163200, x: 148800)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)
Coordinates:
  * y        (y) float64 -9.6e+05 -9.6e+05 -9.600e+05 ... -4.224e+06 -4.224e+06
  * x        (x) float64 1.056e+06 1.056e+06 1.056e+06 ... 4.032e+06 4.032e+06
Attributes:
    transform:      (20.0, 0.0, 1056000.0, 0.0, -20.0, -960000.0)
    crs:            +init=epsg:6933
    res:            (20.0, 20.0)
    is_tiled:       1
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area

In [8]:
write_cog(aez.squeeze(),
          'WDPA_southern.tif',
          overwrite=True)

PosixPath('WDPA_eastern.tif')